<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-29 10:26:50
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  10.00 K
Current:  1.33 C
-------------------
Today PnL: -16.99 K (-0.13%)
Current PnL: -18.48 L (-12.88%)
CY Booked + Current PnL: -10.40 L (-7.25%)
-------------------
Total profit:  2.54 L
Total loss:  -21.02 L
-------------------
Total Booked + Current PnL: 16.58 L (14.23%)
Total Booked PnL: 35.06 L (30.08%)
Curr Year Booked PnL: 8.08 L (6.09%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 76.69 L (57.8%)
Deployed:  1.17 C
Current:  1.33 C
CAGR/XIRR %: 8.37%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.00,-220.50,H-SC,12.06,86892.0,-8424.0,8463.0,0.15,-8.84,9.74,0.04,161.0,-1.00,0.66,71.38,OX40N,NTT,MISC
39,HINDALCO,651.95,761.55,-20.73,H-LC,5.36,109776.0,5464.0,12075.0,-0.47,5.24,11.00,16.81,10.0,0.45,0.83,22.13,X5K,ATH,METALS
58,MEDANTA,1087.93,1486.00,24.88,H-SC,11.36,160150.0,33950.0,12219.0,1.86,26.90,7.63,36.59,154.0,2.78,1.21,42.79,XY24,NTT,HEALTHCARE
4,APOLLOHOSP,6904.43,8285.00,-18.40,H-LC,6.68,111180.0,7614.0,13097.0,0.75,7.35,11.78,20.00,14.0,0.58,0.84,22.46,X40N,BTT,HEALTHCARE
17,BRITANNIA,5190.70,6446.05,12.32,H-LC,9.65,100728.0,7295.0,15301.0,-0.51,7.81,15.19,24.18,35.0,0.48,0.76,22.31,XY25,ATH,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,QUESS,296.83,986.00,-34.99,M-SC,42.11,64780.0,-226.0,151151.0,0.65,-0.35,233.33,232.18,196.0,-0.00,0.49,0.65,XY24,NTT,MISC
14,BERGEPAINT,561.33,680.00,-20.21,H-MC,10.80,226030.0,-1309.0,49365.0,-0.36,-0.58,21.84,21.14,108.0,-0.03,1.70,26.55,XY24,NTT,PAINTS
38,HEROMOTOCO,4311.81,5949.07,-6.19,H-MC,3.14,150899.0,-14.0,57311.0,0.47,-0.01,37.98,37.97,89.0,-0.00,1.14,25.29,AR,ATH,AUTO
59,NATIONALUM,189.63,247.44,-50.15,H-MC,3.55,99788.0,-716.0,31353.0,0.31,-0.71,31.42,30.49,81.0,-0.02,0.75,31.46,MH,ATH,METALS


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,CIPLA,1492.70,1795.0,-18.42,H-LC,1.36,214857.0,10357.0,31068.0,-0.24,5.06,14.46,20.25,7.0,0.33,1.62,14.24,X40N,BTT,PHARMA
56,LTIM,5564.16,7230.2,-5.95,H-LC,2.57,181080.0,-19230.0,79204.0,-0.34,-9.60,43.74,29.94,26.0,-0.24,1.37,25.10,X200,ATH,IT
10,BAJAJHFL,122.26,152.0,-8.98,H-LC,2.76,188912.0,-11594.0,60376.0,-0.01,-5.78,31.96,24.33,30.0,-0.19,1.42,8.94,X40N,BTT,FINANCE
40,HINDUNILVR,2413.81,2723.0,-12.66,H-LC,3.40,273448.0,3101.0,31529.0,-0.00,1.15,11.53,12.81,18.0,0.10,2.06,14.36,XY25,NTT,FMCG
5,ASIANPAINT,2961.56,4250.0,-19.64,H-LC,3.44,159630.0,-41756.0,129364.0,-0.51,-20.73,81.04,43.51,20.0,-0.32,1.20,11.12,X40,BTT,PAINTS


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,20.60,67949.0,-45600.0,85650.0,2.05,-40.16,126.05,35.27,267.0,-0.53,0.51,66.31,XR,NTT,HOTELS
53,KPIGREEN,497.21,731.64,25.90,H-SC,12.67,128759.0,3462.0,55611.0,2.03,2.76,43.19,47.15,143.0,0.06,0.97,61.09,MH,ATH,POWER
51,JIOFIN,310.58,387.00,2.30,H-LC,13.19,219170.0,1764.0,51724.0,1.87,0.81,23.60,24.61,49.0,0.03,1.65,55.83,XY24,BTT,FINANCE
58,MEDANTA,1087.93,1486.00,24.88,H-SC,11.36,160150.0,33950.0,12219.0,1.86,26.90,7.63,36.59,154.0,2.78,1.21,42.79,XY24,NTT,HEALTHCARE
18,BSOFT,628.85,836.99,-18.25,M-SC,12.43,68521.0,-41528.0,77949.0,1.63,-37.74,113.76,33.10,216.0,-0.53,0.52,13.86,XR,ATH,IT


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,EASEMYTRIP,18.12,26.4,1.63,M-SC,19.78,97775.0,-79040.0,159833.0,-2.05,-44.70,163.47,45.70,195.0,-0.49,0.74,0.00,XY24,NTT,TRAVEL
32,ENRIN,1377.95,3176.7,NaN,NaN,2.45,74382.0,39933.0,5036.0,-1.39,115.92,6.77,130.54,1.0,7.93,0.56,13.74,AR,ATH,ELECTRICAL
68,SAMMAANCAP,170.35,326.0,-163.64,M-SC,4.34,73704.0,-28506.0,121899.0,-1.30,-27.89,165.39,91.37,210.0,-0.23,0.56,19.65,XY25,NTT,FINANCE
87,UNIONBANK,123.87,163.0,-13.71,M-LC,17.19,149822.0,8982.0,35508.0,-1.25,6.38,23.70,31.59,72.0,0.25,1.13,34.56,XY24,NTT,BANKS
63,RAJESHEXPO,517.65,518.0,1782.58,M-SC,3.82,51357.0,-85820.0,85915.0,-1.18,-62.56,167.29,0.07,239.0,-1.00,0.39,27.73,OX40N,NTT,JEWELLERY


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.0,-220.50,H-SC,12.06,86892.0,-8424.0,8463.0,0.15,-8.84,9.74,0.04,161.0,-1.00,0.66,71.38,OX40N,NTT,MISC
78,SYMPHONY,1306.42,1306.0,-31.84,M-SC,9.54,145148.0,-25993.0,25938.0,-0.07,-15.19,17.87,-0.03,194.0,-1.00,1.09,5.04,OX40N,NTT,DURABLES
86,UJJIVANSFB,52.77,53.0,36.52,M-SC,21.73,117963.0,-24516.0,25138.0,-0.23,-17.21,21.31,0.44,249.0,-0.98,0.89,39.23,OX40N,NTT,BANKS
85,TTKPRESTIG,769.29,770.0,-9.35,H-SC,12.04,82792.0,-17985.0,18082.0,-0.78,-17.85,21.84,0.09,144.0,-0.99,0.62,7.74,OX40N,NTT,DURABLES
80,TATAELXSI,7332.28,7332.0,-19.83,H-MC,1.73,84126.0,-18526.0,18525.0,-0.44,-18.05,22.02,-0.00,100.0,-1.00,0.63,27.88,OX40N,NTT,IT


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3176.70,NaN,NaN,2.45,74382.0,39933.0,5036.0,-1.39,115.92,6.77,130.54,1.0,7.93,0.56,13.74,AR,ATH,ELECTRICAL
91,VBL,492.64,672.28,-13.47,H-LC,12.53,262544.0,-2496.0,99137.0,0.26,-0.94,37.76,36.46,2.0,-0.03,1.98,12.12,X40N,ATH,FMCG
82,TCS,3794.03,4998.00,-26.74,H-LC,12.75,238313.0,-57621.0,151519.0,-0.78,-19.47,63.58,31.73,3.0,-0.38,1.80,0.00,X40,BTT,IT
48,INFY,1461.46,2275.00,-16.01,H-LC,6.81,269037.0,7436.0,138177.0,-0.86,2.84,51.36,55.67,5.0,0.05,2.03,9.12,X40,BTT,IT
1,ABB,5551.76,7934.00,-39.96,H-LC,7.44,214344.0,-2175.0,95083.0,-0.89,-1.00,44.36,42.91,6.0,-0.02,1.62,12.04,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,267.99,NaN,NaN,3.20,52676.0,11421.0,6016.0,0.97,27.68,11.42,42.26,197.0,1.90,0.40,24.02,XY24,ATH,IT
58,MEDANTA,1087.93,1486.00,24.88,H-SC,11.36,160150.0,33950.0,12219.0,1.86,26.90,7.63,36.59,154.0,2.78,1.21,42.79,XY24,NTT,HEALTHCARE
32,ENRIN,1377.95,3176.70,NaN,NaN,2.45,74382.0,39933.0,5036.0,-1.39,115.92,6.77,130.54,1.0,7.93,0.56,13.74,AR,ATH,ELECTRICAL


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,KPIGREEN,497.21,731.64,25.90,H-SC,12.67,128759.0,3462.0,55611.0,2.03,2.76,43.19,47.15,143.0,0.06,0.97,61.09,MH,ATH,POWER
58,MEDANTA,1087.93,1486.00,24.88,H-SC,11.36,160150.0,33950.0,12219.0,1.86,26.90,7.63,36.59,154.0,2.78,1.21,42.79,XY24,NTT,HEALTHCARE
20,CAMS,3643.00,5226.82,-2.58,H-SC,0.72,109500.0,7496.0,36847.0,0.38,7.35,33.65,43.48,122.0,0.20,0.83,25.36,X40N,ATH,MISC
26,DABUR,505.20,735.00,-6.82,H-MC,4.50,201786.0,4758.0,84871.0,-0.88,2.41,42.06,45.49,96.0,0.06,1.52,15.77,XY24,BTT,FMCG
15,BLUESTARCO,1646.70,2326.38,-7.42,H-SC,10.27,173520.0,8850.0,59118.0,-0.20,5.37,34.07,41.28,119.0,0.15,1.31,13.79,X40N,ATH,AC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3176.70,NaN,NaN,2.45,74382.0,39933.0,5036.0,-1.39,115.92,6.77,130.54,1.0,7.93,0.56,13.74,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1219.98,2270.00,-38.30,M-SC,8.41,103725.0,12226.0,66529.0,0.24,13.36,64.14,86.07,205.0,0.18,0.78,50.05,XR,NTT,DURABLES
45,INDIAMART,2327.09,4850.92,-48.45,H-SC,12.79,134132.0,10796.0,122972.0,-1.00,8.75,91.68,108.45,118.0,0.09,1.01,33.06,AR,ATH,MISC
53,KPIGREEN,497.21,731.64,25.90,H-SC,12.67,128759.0,3462.0,55611.0,2.03,2.76,43.19,47.15,143.0,0.06,0.97,61.09,MH,ATH,POWER
95,WIPRO,243.46,420.00,-9.67,M-LC,5.65,154380.0,3435.0,106013.0,-0.42,2.28,68.67,72.51,56.0,0.03,1.16,8.44,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,94.95,NaN,NaN,6.53,19644.0,-29949.0,1149.0,0.53,-60.39,5.85,-58.07,204.0,-26.07,0.15,31.74,XY24,ATH,MISC
74,SIS,477.00,477.00,2281.87,M-SC,5.82,64082.0,-17008.0,17007.0,0.45,-20.97,26.54,0.00,238.0,-1.00,0.48,29.87,OX40N,NTT,MISC
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,20.60,67949.0,-45600.0,85650.0,2.05,-40.16,126.05,35.27,267.0,-0.53,0.51,66.31,XR,NTT,HOTELS
92,VIPIND,488.80,489.00,-220.50,H-SC,12.06,86892.0,-8424.0,8463.0,0.15,-8.84,9.74,0.04,161.0,-1.00,0.66,71.38,OX40N,NTT,MISC
3,ALKYLAMINE,2347.98,4546.37,0.87,H-SC,20.00,93955.0,-7008.0,101537.0,1.29,-6.94,108.07,93.63,147.0,-0.07,0.71,39.94,SR,ATH,CHEMICALS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,267.99,NaN,NaN,3.20,52676.0,11421.0,6016.0,0.97,27.68,11.42,42.26,197.0,1.90,0.40,24.02,XY24,ATH,IT
58,MEDANTA,1087.93,1486.00,24.88,H-SC,11.36,160150.0,33950.0,12219.0,1.86,26.90,7.63,36.59,154.0,2.78,1.21,42.79,XY24,NTT,HEALTHCARE
33,GREENPANEL,375.16,537.00,250.13,M-SC,9.43,156404.0,-26674.0,105651.0,0.08,-14.57,67.55,43.14,234.0,-0.25,1.18,47.12,XY24,NTT,MISC
16,BLUSPRING,226.45,94.95,NaN,NaN,6.53,19644.0,-29949.0,1149.0,0.53,-60.39,5.85,-58.07,204.0,-26.07,0.15,31.74,XY24,ATH,MISC
72,SHALBY,261.39,327.00,966.99,M-SC,9.66,145128.0,-36015.0,81489.0,-0.10,-19.88,56.15,25.10,228.0,-0.44,1.09,16.31,XY24,NTT,HEALTHCARE


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.48
1,25,41.23
2,50,70.15


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    41.69
LC    35.18
MC    22.02
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.87
X40N     14.64
X40      12.52
XY25     10.78
XR        9.09
OX40N     8.39
AR        8.21
X5K       2.25
MH        1.72
X200      1.37
SR        1.16
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    26.95
H-SC    25.28
H-MC    19.04
M-SC    15.01
M-LC     7.18
M-MC     2.64
L-SC     1.40
L-LC     1.05
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.16,-4.21,39.07
IT,11.74,-16.80,74.35
FINANCE,8.31,-18.25,61.15
BANKS,7.54,-13.98,60.49
MISC,6.09,-19.29,75.00
PAINTS,5.62,-17.05,40.12
ELECTRICAL,5.37,-3.41,52.41
HEALTHCARE,4.69,2.93,22.85
AUTO,4.44,-14.97,64.99


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2758100.0,25
XR,1086152.0,13
AR,1056691.0,10
X40,792704.0,10
X40N,608747.0,12
XY25,509456.0,8
OX40N,359712.0,11
SR,189862.0,2
X5K,141589.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2471547.0,26
M-SC,1640759.0,20
H-LC,1291863.0,20
H-MC,1178900.0,15
M-LC,426298.0,5
M-MC,328104.0,2
L-SC,212780.0,3
L-MC,56907.0,1
L-LC,49822.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,958684.0,7
M-SC,XY24,783547.0,7
H-SC,AR,527422.0,3
H-LC,X40,513123.0,6
H-MC,XY24,455612.0,4
H-SC,XR,362366.0,4
M-MC,XY24,328104.0,2
H-LC,X40N,309256.0,6
M-SC,XR,287688.0,4


In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
